In [5]:

# file = open("D:/pycharm/untitled1/Output.html",'r').read()
# soup = BeautifulSoup(file)
import os
import json
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd

import datetime as datetime   




def parse_url(url,writer):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    script = soup.body.text
    script = script.replace("\n", "")
    re_match = re.match('.*race\.stateData = (.*);race\.path', script)
    str_json = re_match.group(1)
    j = json.loads(str_json)
    polls = j['polls']
    date = []
    pollster = []
    grade = []
    sample = []
    weight = []
    clinton = []
    trump = []
    johnson = []
    leader = []

    adjusted = []
    for poll in polls:
        d1=datetime.datetime.strptime(str(poll['endDate']), "%Y-%m-%d")
        d2=datetime.datetime.today()
        if (abs(d1-d2).days)<30:
            date.append(str(poll['startDate']) + ' -> '+str( poll['endDate']))
            pollster.append(poll['pollster'])
            grade.append(poll['pollsterRating'])
            sample.append( poll['sampleSize'])
            weight.append(poll['weight']['polls'])

            clinton_flag = False
            trump_flag=False
            johnson_flag=False
            for poll_answer in poll['votingAnswers']:
                pct=str(poll_answer['pct'])
                if poll_answer['choice'] == 'Trump':
                    trump.append(pct)
                    trump_flag = True
                elif poll_answer['choice'] == 'Clinton':
                    clinton.append(pct)
                    clinton_flag = True
                else:
                    johnson.append(pct)
                    johnson_flag = True

            if not johnson_flag:
                johnson.append('')
            if not clinton_flag:
                clinton.append('')
            if not trump_flag:
                trump.append('')

            leader.append('?')
            adjusted.append('?')
        

    save_csv(writer,url, adjusted, clinton, date, grade, johnson, leader, pollster, sample, trump, weight)


def save_csv(writer,url, adjusted, clinton, date, grade, johnson, leader, pollster, sample, trump, weight):
    df = pd.DataFrame(date, columns=['date'])
    df['pollster'] = pollster
    df['grade'] = grade
    df['Sample'] = sample
    df['weight'] = weight
    df['Clinton'] = clinton
    df['Trump'] = trump
    #df['johnson'] = johnson
    df['leader'] = leader
    df['adjusted'] = adjusted
    s = urlparse(url)
    filename = os.getcwd() + "/" + s.path.split('/')[-2] + '.csv'
    #df.to_csv(filename)
    df.to_excel(writer, s.path.split('/')[-2])



    

In [6]:
url = 'http://projects.fivethirtyeight.com/2016-election-forecast/georgia/'
rootparsed = urlparse(url)
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")
links_to_watch = soup.find("ul", attrs={"class": "nav-list all"})
links_to_watch.find_all('li')
writer = pd.ExcelWriter('StatePoll'+str(datetime.date.today())+'.xlsx')
for link in links_to_watch:
    parse_url(rootparsed.scheme+"://" + rootparsed.netloc+ link.a['href'],writer)

    
writer.save()


In [ ]:
datetime.date.today()
